In [1]:
## NOTE: the error is totally screwed up in 2015...
## generate a new dataset starting in 2010 and see if the error for 2015 is similarly big?

In [2]:
import pandas as pd
import numpy as np
from data_functions import *
from sklearn.metrics import log_loss, accuracy_score

full_df = pd.read_csv('match_data_constructed/match_df_6_29_2018')
# current_df = pd.read_csv('match_data_constructed/current_match_df_5_30_2018')
# TO DO: compare online elo ratings with current_df as sanity check
# ISSUE: tny_ids are not constant from year to year...
# still, replace missing tny_averages with overall average...
# TO DO: fix generate_EM_stats()

In [24]:
sub = full_df[(full_df['p0_name']=='Nick Kyrgios') & (full_df['p1_name']=='Mikhail Youzhny')]
sub[[]]

,p0_52_rpt,p0_52_rwon,p0_r_pct,p1_52_rpt,p1_52_rwon,p1_r_pct
13819,1323.0,424.0,0.320484,2947.0,1145.0,0.388531


In [9]:
# df = full_df[(full_df['match_year']<=2015) & (full_df['match_year']>=2010)]
df = full_df[(full_df['match_year'] == 2014)]
df = df[df['tny_name']!='ATP Challenger Tour Finals CH']
df = df[df['tny_name']!='Davis Cup']
s_pred = np.concatenate([df['avg_52_s'], df['avg_52_s']], axis=0)
s_true = np.concatenate([df['p0_s_pct_obsv'], df['p1_s_pct_obsv']], axis=0)
print np.mean((s_true - s_pred)**2)**.5, np.mean(s_true - s_pred)

0.0919536106708 0.00435048250394


In [10]:
sub = df[(df['p0_name']=='Kevin Anderson') & (df['p1_name']=='Fernando Verdasco')]
sub[['tny_name','p0_52_svpt', 'p1_52_svpt']]

,tny_name,p0_52_svpt,p1_52_svpt


In [11]:
sr_cols = ['s_kls','s_kls_EM','s_adj_kls','s_adj_kls_EM','s_kls_elo']
for col in sr_cols:
    print col
    s_pred = np.concatenate([df['p0_'+col], df['p1_'+col]], axis=0)
    s_true = np.concatenate([df['p0_s_pct_obsv'], df['p1_s_pct_obsv']], axis=0)
    print np.mean((s_true - s_pred)**2)**.5

s_kls
0.0847189848749
s_kls_EM
0.0824909631948
s_adj_kls
0.082732444118
s_adj_kls_EM
0.0818760437124
s_kls_elo
0.0806206131889


In [5]:
prob_cols = [u'match_prob_kls', u'match_prob_kls_EM', u'match_prob_adj_kls', \
             u'match_prob_adj_kls_EM', u'elo_prob']
for col in prob_cols:
    print col
    print 'accuracy score: ', accuracy_score(1 - df['winner'], df[col] >=.5)
    print 'log loss: ',log_loss(1 - df['winner'], df[col])
    

match_prob_kls
accuracy score:  0.648713826367
log loss:  0.647934936125
match_prob_kls_EM
accuracy score:  0.653135048232
log loss:  0.615573040828
match_prob_adj_kls
accuracy score:  0.674839228296
log loss:  0.631434512664
match_prob_adj_kls_EM
accuracy score:  0.675241157556
log loss:  0.619100180172
elo_prob
accuracy score:  0.690514469453
log loss:  0.588863185483


In [53]:
df.shape

(2488, 74)

In [35]:
# view performance, segmented by tournament
tourneys = full_df[full_df['match_year']==2011]
tourneys = list(set(tourneys['tny_name']))
tourney_scores = []

for tourney in tourneys:
    df = full_df[(full_df['match_year']==2011) & (full_df['tny_name']==tourney)]
    s_pred = np.concatenate([df['p0_s_adj_kls_EM'], df['p1_s_adj_kls_EM']], axis=0)
    s_pred_elo = np.concatenate([df['p0_s_kls_elo'], df['p1_s_kls_elo']], axis=0)
    s_true = np.concatenate([df['p0_s_pct_obsv'], df['p1_s_pct_obsv']], axis=0)
    tourney_scores.append((tourney, np.mean((s_true - s_pred)**2)**.5, \
#                            np.mean((s_true - s_pred_elo)**2)**.5, \
                           len(df)))
# log_loss(1-sub_df['winner'], sub_df['match_prob_kls_EM'])
print sorted(tourney_scores, key=lambda x:x[1])[::-1]

[('Davis Cup', 0.63285121741060535, 299), ('ATP Challenger Tour Finals CH', 0.59205902125006304, 15), ('Beijing', 0.14061271483842591, 31), ('Bucharest', 0.11060407549433449, 26), ('Costa Do Sauipe', 0.10339703435259952, 27), ('Bastad', 0.098218451938211201, 25), ('Chennai', 0.096892200446861046, 31), ('Newport', 0.096211802135544769, 30), ('Houston', 0.095101358982427095, 27), ('Brisbane', 0.094901960149957013, 31), ('Sydney', 0.093703060111054351, 26), ('St. Petersburg', 0.092514638108521521, 30), ('Metz', 0.091956588183749396, 26), ('Zagreb', 0.091377694857813607, 25), ('Johannesburg', 0.091281644206172782, 31), ('Washington', 0.091175582121277965, 43), ('Estoril', 0.09003675832871863, 24), ('Atlanta', 0.089634404855847491, 27), ('Kitzbuhel', 0.08960325236822321, 26), ('Stuttgart', 0.08933506844402013, 31), ('Los Angeles', 0.089004295534191036, 27), ('Doha', 0.088726430289891273, 31), ('Paris Masters', 0.087546050471564399, 44), ('Madrid Masters', 0.087529357775440797, 52), ('Dubai'

In [11]:
# print 'avg_52_s'
# s_pred = np.concatenate([df['avg_52_s'], df['avg_52_s']], axis=0)
# s_true = np.concatenate([df['p0_s_pct_obsv'], df['p1_s_pct_obsv']], axis=0)
# print np.std(s_true - s_pred), np.mean(s_true - s_pred)

In [36]:
np.mean(s_true - s_pred)

-0.03742178285002018

In [51]:
a = concat_data(1968, 2018, 'atp')
a = format_match_df(a, 'atp')

t_name, m_year, t_id = 'Geneva', 2014, '2014-322'
a[(a['match_year']==m_year)]

In [44]:
t_name = 'Brisbane'
m_year = 2016
sub = df[df['tny_name']==t_name]
sub = sub[sub['match_year']==m_year]
sub[['match_year', 'tny_id', 'tny_stats']].groupby('tny_id').count()

,match_year,tny_stats
tny_id,,
2016-M020,27,27


In [67]:
df[(df['tny_name']=='Cincinnati Masters') & (df['match_year']==2015)]

,tny_id,tny_name,surface,draw_size,tny_date,match_num,w_name,l_name,score,best_of,...,elo_diff,sf_elo_diff,p0_s_pct_JS,p1_s_pct_JS,p0_r_pct_JS,p1_r_pct_JS,p0_52_s_adj_JS,p1_52_s_adj_JS,p0_52_r_adj_JS,p1_52_r_adj_JS
2129,2015-422,Cincinnati Masters,Hard,56,2015-08-16,1,Benoit Paire,Gilles Muller,7-6 7-6,3,...,-70.140220,-166.967054,0.631329,0.674199,0.378020,0.322283,0.013324,0.086554,0.052913,-0.014510
2130,2015-422,Cincinnati Masters,Hard,56,2015-08-16,2,Fernando Verdasco,Jo Wilfried,4-6 6-3 6-4,3,...,-188.011765,-183.762474,0.651035,0.676275,0.357261,0.355530,0.053831,0.093527,0.031402,0.047752
2131,2015-422,Cincinnati Masters,Hard,56,2015-08-16,3,David Goffin,Yen Hsun,5-7 6-3 6-4,3,...,176.636910,170.816212,0.648477,0.645149,0.381055,0.351706,0.040466,0.030241,0.072604,0.025800
2132,2015-422,Cincinnati Masters,Hard,56,2015-08-16,4,Ivo Karlovic,Gilles Simon,6-4 6-7 6-3,3,...,110.857036,98.839916,0.634155,0.726568,0.384512,0.308524,0.024290,0.151007,0.079019,-0.035418
2133,2015-422,Cincinnati Masters,Hard,56,2015-08-16,5,Martin Klizan,Dominic Thiem,7-6 3-6 7-6,3,...,-68.617062,-49.335363,0.623800,0.648289,0.356055,0.354096,0.005965,0.041022,0.029572,0.029038
2134,2015-422,Cincinnati Masters,Hard,56,2015-08-16,6,Borna Coric,Alexander Zverev,7-5 3-6 7-6,3,...,13.073215,121.593802,0.632621,0.638164,0.358185,0.344760,0.015486,0.011376,0.041143,0.008971
2135,2015-422,Cincinnati Masters,Hard,56,2015-08-16,7,Bernard Tomic,Sergiy Stakhovsky,6-4 6-3,3,...,134.072880,195.625039,0.656196,0.641616,0.342035,0.356442,0.044954,0.024202,0.015874,0.033502
2136,2015-422,Cincinnati Masters,Hard,56,2015-08-16,8,Jared Donaldson,Nicolas Mahut,7-5 6-7 6-2,3,...,266.473846,104.458716,0.643708,0.646907,0.353270,0.347949,0.025782,-0.006366,0.018553,0.002687
2137,2015-422,Cincinnati Masters,Hard,56,2015-08-16,9,Jerzy Janowicz,Gael Monfils,6-4 7-5,3,...,-272.222705,-179.625849,0.636603,0.658621,0.349124,0.371610,0.018935,0.063013,0.005018,0.055607
2138,2015-422,Cincinnati Masters,Hard,56,2015-08-16,10,Sam Querrey,John Isner,6-3 7-6,3,...,127.024053,185.309024,0.705726,0.666776,0.309753,0.345871,0.119907,0.066919,-0.030593,0.017798


In [57]:
df[(df['tny_stats']==0) & (df['match_year']==2016)].groupby('tny_name').count()

,tny_id,surface,draw_size,tny_date,match_num,w_name,l_name,score,best_of,w_svpt,...,elo_diff,sf_elo_diff,p0_s_pct_JS,p1_s_pct_JS,p0_r_pct_JS,p1_r_pct_JS,p0_52_s_adj_JS,p1_52_s_adj_JS,p0_52_r_adj_JS,p1_52_r_adj_JS
tny_name,,,,,,,,,,,,,,,,,,,,,
Acapulco,30,30,30,30,30,30,30,30,30,30,...,30,30,30,30,30,30,30,30,30,30
Antwerp,27,27,27,27,27,27,27,27,27,27,...,27,27,27,27,27,27,27,27,27,27
Beijing,30,30,30,30,30,30,30,30,30,30,...,30,30,30,30,30,30,30,30,30,30
Brisbane,27,27,27,27,27,27,27,27,27,27,...,27,27,27,27,27,27,27,27,27,27
Chengdu,27,27,27,27,27,27,27,27,27,27,...,27,27,27,27,27,27,27,27,27,27
Cincinnati Masters,52,52,52,52,52,52,52,52,52,52,...,52,52,52,52,52,52,52,52,52,52
Davis Cup,237,237,237,237,237,237,237,237,237,233,...,237,237,237,237,237,237,237,237,237,237
Indian Wells Masters,92,92,92,92,92,92,92,92,92,92,...,92,92,92,92,92,92,92,92,92,92
Los Cabos,25,25,25,25,25,25,25,25,25,25,...,25,25,25,25,25,25,25,25,25,25


In [30]:
SCRIPT_PATH = '/Users/jacobgollub/Desktop/tennis_probs_live/match_data'
TOUR = 'atp'
START_YEAR = 2015
CURRENT_YEAR = 2018
DATE = '5_30_{}'.format(CURRENT_YEAR)

import sys
sys.path.insert(0,SCRIPT_PATH)

In [5]:
len(sub)

1915

In [4]:
sub = df[df['tny_stats']==0]
sub.groupby(by=['tny_name'])['surface'].count()

tny_name
Acapulco                   30
Antalya                    25
Antwerp                    27
Beijing                    30
Brisbane                   27
Budapest                   25
Challenger Tour Finals     13
Chengdu                    27
Cincinnati Masters         52
Davis Cup                 894
Eastbourne                 27
Estoril                    27
Geneva                     27
Indian Wells Masters       92
Istanbul                   26
Los Cabos                  25
Lyon                       27
Madrid Masters             55
Miami Masters              89
Moscow                     23
Olympics                   61
Quito                      27
Rio De Janeiro             30
Rio de Janeiro             27
Rome Masters               53
Sofia                      26
St. Petersburg             26
Sydney                     24
Washington                 46
s-Hertogenbosch            27
Name: surface, dtype: int64

In [31]:
match_df = concat_data(1968, 2019, TOUR)
ret_strings = ('ABN','DEF','In Progress','RET','W/O',' RET',' W/O','nan','walkover')
abd_strings = ('abandoned','ABN','ABD','DEF','def','unfinished','Walkover')
match_df = format_match_df(match_df,TOUR,ret_strings=ret_strings,abd_strings=abd_strings)
start_ind = match_df[match_df['match_year']>=START_YEAR-1].index[0]

In [71]:
# NOTE: need to ignore NAN values in tny_stats, 52_stats, everything!!!
sub_df = match_df[ (match_df['match_year']==2013)]
s_won = np.sum(sub_df['w_swon']) + np.sum(sub_df['l_swon'])
s_pt =  np.sum(sub_df['w_svpt']) + np.sum(sub_df['l_svpt'])
s_won/s_pt

0.6318865929461523

In [25]:
cols = [u'tourney_id', u'tourney_name', u'surface', u'draw_size', u'tourney_date', u'match_num',\
       u'winner_name', u'loser_name', u'score', u'best_of', u'w_svpt', u'w_1stIn', u'w_1stWon',\
       u'w_2ndWon',  u'l_1stWon', u'l_2ndWon', u'l_SvGms']

In [1]:
# modify this for your own path
SCRIPT_PATH = '/Users/jacobgollub/Desktop/tennis_probs_live/match_data'
TOUR = 'atp'
CURRENT_YEAR = 2018
DATE = '5_14_{}'.format(CURRENT_YEAR)

import sys
sys.path.insert(0,SCRIPT_PATH)
from helper_functions import *
from data_functions import *
pd.options.mode.chained_assignment = None

if __name__=='__main__':
    print 'main'
    df = concat_data(1968, 2019, TOUR)

    ret_strings = ('ABN','DEF','In Progress','RET','W/O',' RET',' W/O','nan','walkover')
    abd_strings = ('abandoned','ABN','ABD','DEF','def','unfinished','Walkover')
    df = format_match_df(df,TOUR,ret_strings=ret_strings,abd_strings=abd_strings)


main


In [2]:
current_elo_ratings = get_current_elo(df, 1)
current_52_stats = get_current_52_stats(df, CURRENT_YEAR) # now, add EM normalization
current_df = current_elo_ratings.merge(current_52_stats, on='player')
current_df = generate_EM_stats(current_df, cols=['52_s_pct','52_r_pct'])

52_s_pct 0.605451686146
52_r_pct 0.349674134798


In [12]:
print np.sum(current_df['52_swon'])/np.sum(current_df['52_svpt'])
print 1 - np.sum(current_df['52_rwon'])/np.sum(current_df['52_rpt'])

0.636364072258
0.636364072258


In [3]:
current_elo_ratings.merge(current_52_stats, on='player')

,player,elo,52_swon,52_svpt,52_rwon,52_rpt
0,Novak Djokovic,2420.335276,1646.0,2511.0,1111.0,2585.0
1,Roger Federer,2399.329427,2910.0,3990.0,1810.0,4580.0
2,Andy Murray,2320.988483,1228.0,1942.0,790.0,1919.0
3,Rafael Nadal,2318.476845,3172.0,4514.0,2179.0,4990.0
4,Juan Martin,2185.581151,2817.0,4219.0,1626.0,4423.0
5,Kei Nishikori,2175.221016,1611.0,2517.0,1052.0,2620.0
6,Grigor Dimitrov,2152.511319,3197.0,4751.0,1802.0,4855.0
7,Nick Kyrgios,2144.663410,2459.0,3483.0,1249.0,3628.0
8,Alexander Zverev,2128.472428,3736.0,5562.0,2063.0,5564.0
9,Stanislas Wawrinka,2122.075679,1723.0,2649.0,916.0,2519.0
